## Data wrangling quiz 

In [1]:
import findspark
findspark.init('/home/nikhil/spark-2.4.4-bin-hadoop2.7/')
from pyspark.sql import SparkSession

In [2]:
spark=SparkSession.builder.appName('Data Wrangling quiz').getOrCreate()

In [233]:
df=spark.read.json('data/sparkify_log_small.json')

In [234]:
df.describe()

DataFrame[summary: string, artist: string, auth: string, firstName: string, gender: string, itemInSession: string, lastName: string, length: string, level: string, location: string, method: string, page: string, registration: string, sessionId: string, song: string, status: string, ts: string, userAgent: string, userId: string]

## Question 1
Which page did user id "" Not visit?

In [244]:
no_duplicates=df.drop_duplicates()
no_duplicates.select("page").where(no_duplicates.userId=="").count()

336

In [265]:
empty_user_pages

[Row(page='Home'), Row(page='About'), Row(page='Login'), Row(page='Help')]

## 2 Reflect
what type of user does the empty string user id most likely refer to?

In [268]:
df.select("page").where(df['userId']=="").dropDuplicates().show()

+-----+
| page|
+-----+
| Home|
|About|
|Login|
| Help|
+-----+



## question3 
#### How many female users do we have in the data set?

In [278]:
df.select(['userId']).dropDuplicates().filter(df["gender"]=="F").count()

462

In [58]:
df.select(['userId','gender']).dropDuplicates().where(df['gender']=='F').count()

462

+------+------+-------------+
|userId|gender|count(gender)|
+------+------+-------------+
|  2902|     F|            2|
|  1872|     M|            1|
|   838|     F|            1|
|   314|     M|           23|
|  1941|     M|            2|
|  1545|     M|            1|
|   577|     F|            2|
|  2122|     F|            6|
|  2049|     M|            1|
|  2967|     M|            2|
|  1887|     F|            9|
|  1661|     F|            2|
|  1403|     M|            2|
|  2546|     F|            2|
|  1866|     F|            7|
|  1162|     F|            4|
|  1232|     M|          180|
|  2675|     F|            7|
|  1865|     M|          137|
|  2933|     M|            1|
+------+------+-------------+
only showing top 20 rows



## Question 4
##### How many songs were played from the most played artist?

In [295]:
most_player=df.where(df['page']=="NextSong").groupBy(['artist']).count().sort('count',ascending=False)

song_count=df.select(df['song']).where(df.page=="NextSong").where(df["artist"]==most_player.collect()[0]['artist'])
song_count.describe('song').show()

+-------+--------------------+
|summary|                song|
+-------+--------------------+
|  count|                  83|
|   mean|                null|
| stddev|                null|
|    min|A Rush Of Blood T...|
|    max|                 Yes|
+-------+--------------------+



In [82]:
song_count=df.select('song').where(df.page=="NextSong").where(df.artist==most_played_artist)

In [84]:
song_count.describe().show()

+-------+--------------------+
|summary|                song|
+-------+--------------------+
|  count|                  83|
|   mean|                null|
| stddev|                null|
|    min|A Rush Of Blood T...|
|    max|                 Yes|
+-------+--------------------+



## Question 5 
#### How many songs do users listen to on average between visting our home page?please round your answer to the closest integer.

In [98]:
from pyspark.sql.types import IntegerType
from pyspark.sql.functions import format_number,min,max,mean,desc,col,udf,sum as Fsum
from pyspark.sql import Window

In [99]:
flag=udf(lambda x:1 if x=="Home" else 0,IntegerType())

In [100]:
windowval=Window.partitionBy("userId").orderBy(desc('ts')).rangeBetween(Window.unboundedPreceding,0)

In [101]:
user_log_valid=df.filter((df.page=="NextSong")|(df.page=="Home")).select('userId','page','ts').withColumn('Visited_home',flag(col("page"))).withColumn("phase",Fsum('visited_home').over(windowval))

In [219]:
user_log_valid.where(user_log_valid.page=='NextSong').groupBy('userID','phase').agg({'phase':'count'}).agg({'count(phase)':'sum'}).show()

+-----------------+
|sum(count(phase))|
+-----------------+
|             8347|
+-----------------+



In [189]:
df= user_log_valid.toPandas()

In [223]:
f=df[df['page']=='NextSong'].groupby('userId','phase').agg(['count'])

ValueError: No axis named phase for object type <class 'pandas.core.frame.DataFrame'>

KeyError: 'count'

In [225]:
f=df[df['page']=='NextSong'].groupby(['userId','phase'])['phase'].agg(['count','mean'])

In [226]:
sum(f['count'])

8347

In [227]:
sum(f['count'])/len(f.index)

6.898347107438017

In [215]:
f

sum  mean
userId phase           
10     0        0     0
100    1        1     1
       2        4     2
       3        6     3
1000   0        0     0
...           ...   ...
986    1        1     1
988    0        0     0
989    0        0     0
998    0        0     0
999    0        0     0

[1210 rows x 2 columns]